# AI model for training (shark fin) artifacts on GOES-R

[How to Train an Image Classifier in PyTorch and use it to Perform Basic Inference on Single Images](https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5)

[TRAINING A CLASSIFIER](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)


In [25]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler

In [26]:
nr.seed = 42
MyTrainData = []
MyTestData = []

In [46]:
data_dir = '/scratch/adomakor412/mydataset3/'
def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                      ])
    ind_train = np.floor((1-valid_size) * len(datadir))
    img_train = datasets.ImageFolder(datadir, transform=train_transforms)
    train_shuffle = np.array(img_train.imgs)
    np.random.shuffle(train_shuffle)
    
    ind_test = np.floor((valid_size) * len(datadir))
    img_test = datasets.ImageFolder(datadir, transform=test_transforms)
    test_shuffle = np.array(img_test.imgs)
    np.random.shuffle(test_shuffle)
    
    
    train_data = np.copy(train_shuffle)
    test_data = np.copy(test_shuffle)
    
    for path in train_data:
        MyTrainData.append(path[0])
    for path in test_data:
        MyTestData.append(path[0])
    
# #     train_data = datasets.ImageFolder(datadir,       
# #                     transform=train_transforms)
# #     test_data = datasets.ImageFolder(datadir,
# #                     transform=test_transforms)
    num_train=(len(datasets.ImageFolder(datadir, transform = train_transforms)))
    train_idx, test_idx = len(train_data), len(test_data)
    test_sampler = SubsetRandomSampler(ind_test)
    train_sampler = SubsetRandomSampler(ind_train)
#     num_train = len(train_data)
#     indices = list(range(num_train))
#     split = int(np.floor(valid_size * num_train))
#     np.random.shuffle(indices)
#     train_idx, test_idx = indices[split:], indices[:split]
#     train_sampler = SubsetRandomSampler(train_idx)
#     test_sampler = SubsetRandomSampler(test_idx)
#     trainloader = torch.utils.data.DataLoader(train_data,
#                    sampler=train_sampler, batch_size=64)
#     testloader = torch.utils.data.DataLoader(test_data,
#                    sampler=test_sampler, batch_size=64)

    trainloader = torch.utils.data.DataLoader(img_train, sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(img_test, sampler=test_sampler, batch_size=64)
    
    return trainloader, testloader
trainloader, testloader = load_split_train_test(data_dir, .2)

print(trainloader.dataset.classes)

AttributeError: 'numpy.ndarray' object has no attribute 'classes'

In [ ]:
print(MyTrainData)
print(MyTestData)

In [4]:
for data in trainloader.dataset:
    print(data[0].shape, data[1])
    break

torch.Size([3, 224, 224]) 0


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = models.resnet50(pretrained=True)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 10),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
epochs = 100#1
steps = 0
running_loss = 0
print_every = 1#10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        print(steps)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print('REACHED RUNNING LOSS')
        if steps % print_every == 0:
            print('I am working')
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device),labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
#torch.save(model, 'aerialmodel.pth')

1
REACHED RUNNING LOSS
I am working
Epoch 1/100.. Train loss: 2.274.. Test loss: 0.505.. Test accuracy: 0.930
2
REACHED RUNNING LOSS
I am working
Epoch 1/100.. Train loss: 1.322.. Test loss: 0.250.. Test accuracy: 0.930
3
REACHED RUNNING LOSS
I am working
Epoch 1/100.. Train loss: 0.329.. Test loss: 2.753.. Test accuracy: 0.070
4
REACHED RUNNING LOSS
I am working
Epoch 1/100.. Train loss: 3.647.. Test loss: 0.587.. Test accuracy: 0.884
5
REACHED RUNNING LOSS
I am working
Epoch 1/100.. Train loss: 0.400.. Test loss: 0.794.. Test accuracy: 0.930
6
REACHED RUNNING LOSS
I am working
Epoch 1/100.. Train loss: 0.675.. Test loss: 1.957.. Test accuracy: 0.884
7
REACHED RUNNING LOSS
I am working
Epoch 1/100.. Train loss: 0.963.. Test loss: 2.388.. Test accuracy: 0.884
8
REACHED RUNNING LOSS
I am working
Epoch 1/100.. Train loss: 3.205.. Test loss: 2.814.. Test accuracy: 0.884
9
REACHED RUNNING LOSS
I am working
Epoch 1/100.. Train loss: 3.597.. Test loss: 2.994.. Test accuracy: 0.884
10
REACHED

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

#### Took 1 hour to train. Dataset has 100 sharkfins and 889 fillins. Together that is 989 samples. The train to test split is 80% to 20%. Average test accuracy is over 90%!